In [29]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import math
import matplotlib.animation as animation

In [30]:
# ------------------------ COMMON SETUP ------------------------

import numpy as np
import random

# Set the random seed for reproducibility
np.random.seed(42)

# Define the number of cities and generate their coordinates randomly in 2D space
num_cities = 20
cities = np.random.rand(num_cities, 2)

def total_distance(path):
    """
    Calculate the total distance of the path visiting all cities in order.

    Parameters:
    path (list or array-like): An ordered list of city indices representing the travel path.

    Returns:
    float: The total Euclidean distance traveled along the path,
           including returning from the last city back to the first.
    """
    return sum(
        np.linalg.norm(cities[path[i]] - cities[path[(i + 1) % len(path)]])
        for i in range(len(path))
    )

def get_neighbor(path):
    """
    Generate a neighboring path by swapping two cities in the current path.

    Parameters:
    path (list or array-like): The current path (a permutation of city indices).

    Returns:
    list: A new path with two cities swapped, representing a neighboring solution.
    """
    a, b = random.sample(range(len(path)), 2)  # Pick two different indices
    new_path = path.copy()
    new_path[a], new_path[b] = new_path[b], new_path[a]  # Swap the cities
    return new_path


In [31]:
# ------------------------ HILL CLIMBING ------------------------

def run_hill_climbing():
    """
    Perform the Hill Climbing optimization algorithm to find a short path
    through all cities (Traveling Salesman Problem approximation).

    This method starts with a random path and iteratively improves it by
    exploring neighboring solutions. If a neighbor has a shorter total distance,
    it becomes the current solution.

    Returns:
    tuple:
        - best_distance (float): The shortest total distance found.
        - best_path (list): The path (sequence of city indices) corresponding to the shortest distance.
    """
    # Initialize with a random path
    path = list(range(num_cities))
    random.shuffle(path)

    best_path = path.copy()
    best_distance = total_distance(best_path)

    # Try 1000 neighbors and accept better ones
    for _ in range(1000):
        neighbor = get_neighbor(best_path)
        dist = total_distance(neighbor)

        if dist < best_distance:
            best_path = neighbor
            best_distance = dist

    return best_distance, best_path


In [32]:
# ------------------------ SIMULATED ANNEALING ------------------------

def run_simulated_annealing():
    """
    Perform the Simulated Annealing algorithm to approximate the optimal path
    for the Traveling Salesman Problem.

    Simulated Annealing probabilistically accepts worse solutions early on to escape
    local minima. Over time, it reduces the chance of accepting worse solutions,
    gradually 'cooling down' the system.

    Returns:
    tuple:
        - best_distance (float): The shortest total distance found.
        - best_path (list): The path (sequence of city indices) corresponding to the shortest distance.
    """
    # Initial parameters
    initial_temp = 100       # Starting temperature
    cooling_rate = 0.995     # Rate at which temperature cools
    min_temp = 1e-3          # Stop condition (low temperature)

    # Initialize with a random path
    path = list(range(num_cities))
    random.shuffle(path)
    current_path = path.copy()
    current_distance = total_distance(current_path)
    best_path = path.copy()
    best_distance = current_distance

    temperature = initial_temp

    # Annealing process
    while temperature > min_temp:
        # Generate neighboring solution
        neighbor = get_neighbor(current_path)
        neighbor_distance = total_distance(neighbor)
        delta = neighbor_distance - current_distance

        # Accept neighbor if it's better, or probabilistically if it's worse
        if delta < 0 or random.random() < math.exp(-delta / temperature):
            current_path = neighbor
            current_distance = neighbor_distance

            # Update best solution found
            if current_distance < best_distance:
                best_path = current_path
                best_distance = current_distance

        # Cool down
        temperature *= cooling_rate

    return best_distance, best_path


In [33]:
# ------------------------ RUN MULTIPLE TIMES ------------------------

# Lists to store results for Hill Climbing
hc_distances = []  # Final distances
hc_times = []      # Execution times
hc_paths = []      # Best paths found

# Lists to store results for Simulated Annealing
sa_distances = []
sa_times = []
sa_paths = []

# Run both algorithms multiple times (e.g., 5 runs each)
for i in range(5):
    # Run Hill Climbing
    start = time.time()
    d, p = run_hill_climbing()
    t = time.time() - start
    hc_distances.append(d)
    hc_times.append(t)
    hc_paths.append(p)

    # Run Simulated Annealing
    start = time.time()
    d, p = run_simulated_annealing()
    t = time.time() - start
    sa_distances.append(d)
    sa_times.append(t)
    sa_paths.append(p)


In [34]:
# ------------------------ STATS + PLOTTING ------------------------

avg_hc_time = np.mean(hc_times)
avg_sa_time = np.mean(sa_times)
avg_hc_dist = np.mean(hc_distances)
avg_sa_dist = np.mean(sa_distances)

print(f"Avg HC Distance: {avg_hc_dist:.3f}, Avg Time: {avg_hc_time:.4f}s")
print(f"Avg SA Distance: {avg_sa_dist:.3f}, Avg Time: {avg_sa_time:.4f}s")

labels = ['Hill Climbing', 'Simulated Annealing']

# Runtime and Distance Comparison
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.bar(labels, [avg_hc_time, avg_sa_time], color=['blue', 'green'])
plt.ylabel("Avg Runtime (s)")
plt.title("Avg Runtime Comparison")

plt.subplot(1, 2, 2)
plt.bar(labels, [avg_hc_dist, avg_sa_dist], color=['blue', 'green'])
plt.ylabel("Avg Final Distance")
plt.title("Avg Path Distance Comparison")

# Save charts as PNG
plt.tight_layout()
plt.savefig("runtime_distance_comparison.png")
plt.close()

Avg HC Distance: 4.685, Avg Time: 0.0408s
Avg SA Distance: 4.120, Avg Time: 0.0954s


In [35]:
# ------------------------ LINE CHART FOR EACH RUN ------------------------

# Plot Line Charts for Time Comparison
plt.figure(figsize=(10, 5))
plt.plot(range(1, 6), hc_times, label="Hill Climbing", marker='o', color='blue')
plt.plot(range(1, 6), sa_times, label="Simulated Annealing", marker='o', color='green')
plt.xlabel("Run Number")
plt.ylabel("Runtime (seconds)")
plt.title("Runtime Comparison for Each Run")
plt.legend()

# Save line chart for runtime
plt.savefig("runtime_comparison_line_chart.png")
plt.close()

# Plot Line Charts for Distance Comparison
plt.figure(figsize=(10, 5))
plt.plot(range(1, 6), hc_distances, label="Hill Climbing", marker='o', color='blue')
plt.plot(range(1, 6), sa_distances, label="Simulated Annealing", marker='o', color='green')
plt.xlabel("Run Number")
plt.ylabel("Final Distance")
plt.title("Final Distance Comparison for Each Run")
plt.legend()

# Save line chart for distance
plt.savefig("distance_comparison_line_chart.png")
plt.close()

In [36]:
# ------------------------ BEST ROUTE PLOTS ------------------------

def plot_best_route(best_path, algo_name, color, filename):
    """
    Plot and save the best route found by an optimization algorithm.

    Parameters:
    best_path (list): Ordered list of city indices representing the best path.
    algo_name (str): Name of the algorithm (used for the plot title).
    color (str): Line color for the plotted route.
    filename (str): File name to save the plot as a PNG.
    """
    coords = cities[best_path + [best_path[0]]]  # Close the path loop (return to start)

    plt.figure(figsize=(6, 6))
    plt.plot(coords[:, 0], coords[:, 1], marker='o', color=color, lw=2, label='Path')
    plt.scatter(cities[:, 0], cities[:, 1], c='black')  # All city locations

    # Mark start and end points
    plt.scatter(*cities[best_path[0]], c='green', s=100, label='Start')
    plt.scatter(*cities[best_path[-1]], c='red', s=100, label='End')

    # Title and annotations
    plt.title(f"Best Route ({algo_name})\nDistance: {total_distance(best_path):.3f}")
    plt.legend()
    plt.grid(True)

    # Save the figure
    plt.savefig(filename)
    plt.close()

# Identify indices of the best run (shortest distance) for each algorithm
best_hc_index = np.argmin(hc_distances)
best_sa_index = np.argmin(sa_distances)

# Plot and save best routes
plot_best_route(hc_paths[best_hc_index], "Hill Climbing", "blue", "best_route_hc.png")
plot_best_route(sa_paths[best_sa_index], "Simulated Annealing", "green", "best_route_sa.png")


In [37]:
# ------------------------ GIF ANIMATION ------------------------

from matplotlib import animation

def animate_path(path_history, filename, color, title):
    """
    Create and save a GIF animation showing the evolution of a path over time.

    Parameters:
    path_history (list of lists): Sequence of paths representing the algorithm's progress.
    filename (str): The output filename (GIF format).
    color (str): Color used for drawing the path.
    title (str): Title shown on the animation frames.
    """
    # Set up the plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.scatter(cities[:, 0], cities[:, 1], c='black')  # Plot all city locations
    line, = ax.plot([], [], color=color, lw=2)  # Initialize the path line
    title_obj = ax.set_title(title)

    def update(frame):
        """
        Update function for each frame in the animation.

        Parameters:
        frame (int): Frame index.

        Returns:
        tuple: Updated line and title objects.
        """
        path = path_history[frame]
        coords = cities[path + [path[0]]]  # Loop back to start
        line.set_data(coords[:, 0], coords[:, 1])
        title_obj.set_text(f"Step {frame} | Distance: {total_distance(path):.3f}")
        return line, title_obj

    # Create the animation
    ani = animation.FuncAnimation(
        fig, update, frames=len(path_history), interval=200, blit=False
    )

    # Save the animation as a GIF
    ani.save(filename, writer='pillow', fps=5)
    plt.close()

# Create and save GIF for Hill Climbing (shows evolution over 5 runs)
animate_path(hc_paths, "hill_climbing_animation.gif", "blue", "Hill Climbing Path Animation")

# Create and save GIF for Simulated Annealing (shows evolution over 5 runs)
animate_path(sa_paths, "simulated_annealing_animation.gif", "green", "Simulated Annealing Path Animation")

print("Charts and GIFs saved successfully!")


Charts and GIFs saved successfully!
